In [ ]:
# ==========================
# 📦 1. Install Dependencies
# ==========================
!pip install selenium==4.10.0 undetected-chromedriver beautifulsoup4 deep-translator pandas
!apt update && apt install -y wget unzip

# ==============================
# 🧩 2. Install Chrome & Driver
# ==============================
# Install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt --fix-broken install -y

# Install Chromedriver (make sure the version matches Chrome's major version)
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/chromedriver
!chmod +x /usr/local/bin/chromedriver

# =======


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
38 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources

In [ ]:
import time, json, re
import pandas as pd
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# ==========================
# 📄 2. Web Scraping + Translation
# ==========================

BASE_URL = "https://christieathome.com/blog/chinese/page/{}/"
translator = GoogleTranslator(source='en', target='ms')

def get_dynamic_page(url):
    options = uc.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = uc.Chrome(options=options, headless=True)
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.fsri-list")))
    except:
        print(f"Timeout on {url}")
    html = driver.page_source
    driver.quit()
    return html

def get_article_links(page_num):
    html = get_dynamic_page(BASE_URL.format(page_num))
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("ul.fsri-list li a")
    return [a['href'] for a in articles if a.get('href')]

def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def scrape_recipe(url):
    try:
        html = get_dynamic_page(url)
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find("h1").get_text(strip=True)
        content_div = soup.find("div", class_="entry-content")
        if not content_div:
            return None
        paragraphs = content_div.find_all("p")
        content = "\n".join([p.get_text(strip=True) for p in paragraphs if len(p.get_text(strip=True)) > 30])
        if len(content) < 200:
            return None
        title_ms = translator.translate(clean_text(title))
        content_ms = translator.translate(clean_text(content))
        return {
            "title_en": title,
            "title_ms": title_ms,
            "content_en": content,
            "content_ms": content_ms,
            "source_url": url
        }
    except Exception as e:
        print(f"Failed {url}: {e}")
        return None

print("🔍 Scraping...")
all_data, seen = [], set()
for page in range(1, 4):
    for link in get_article_links(page):
        if link not in seen:
            seen.add(link)
            print(f"🔗 {link}")
            recipe = scrape_recipe(link)
            if recipe:
                all_data.append(recipe)
            time.sleep(2)

with open("christie_recipes_ms.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)

pd.DataFrame(all_data).to_csv("christie_recipes_ms.csv", index=False)
print(f"✅ Scraped {len(all_data)} recipes.")



🔍 Scraping...
🔗 https://christieathome.com/blog/string-bean-chicken/
🔗 https://christieathome.com/blog/beef-mushroom-stir-fry/
🔗 https://christieathome.com/blog/honey-garlic-shrimp/
🔗 https://christieathome.com/blog/beef-chow-mein/
🔗 https://christieathome.com/blog/sweet-chili-chicken/
🔗 https://christieathome.com/blog/firecracker-shrimp/
🔗 https://christieathome.com/blog/chicken-lo-mein/
🔗 https://christieathome.com/blog/mapo-tofu/
🔗 https://christieathome.com/blog/honey-garlic-chicken/
🔗 https://christieathome.com/blog/spicy-garlic-bok-choy/
🔗 https://christieathome.com/blog/szechuan-chicken/
🔗 https://christieathome.com/blog/hong-kong-ham-egg-sandwich/
🔗 https://christieathome.com/blog/recipe-coconut-mango-sago-dessert-vegan/
🔗 https://christieathome.com/blog/chinese-spicy-sour-potatoes/
🔗 https://christieathome.com/blog/hong-kong-tomato-macaroni-soup/
🔗 https://christieathome.com/blog/chicken-shrimp-chow-mein/
🔗 https://christieathome.com/blog/air-fryer-sweet-and-sour-tofu/
🔗 https

In [ ]:

# ==========================
# 🧠 3. Convert to SQuAD Format (QA dataset)
# ==========================
qa_dataset = []
for item in all_data:
    context = item["content_ms"]
    title = item["title_ms"]
    for sentence in context.split(".")[:2]:
        if len(sentence.strip()) < 15: continue
        qa_dataset.append({
            "title": title,
            "context": context,
            "question": "",
            "answers": [{"text": sentence.strip(), "answer_start": context.find(sentence.strip())}]
        })

with open("christie_qa_template.json", "w", encoding="utf-8") as f:
    json.dump({"data": qa_dataset}, f, ensure_ascii=False, indent=2)

In [ ]:
# ==========================
# ❓ 4. Generate Questions using T5 model
# ==========================
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it me

In [ ]:

for entry in qa_dataset:
    ans = entry["answers"][0]["text"]
    ctx = entry["context"]
    input_text = f"generate question: {ans} context: {ctx}"
    ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)
    output = model.generate(ids, max_length=64, num_beams=4)
    entry["question"] = tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
with open("christie_qa_squad.json", "w", encoding="utf-8") as f:
    json.dump({"data": qa_dataset}, f, ensure_ascii=False, indent=2)

In [ ]:
# ==========================
# 🔧 5. Train Reader Model (mT5)
# ==========================
!pip install -U transformers datasets evaluate
from datasets import Dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments

with open("christie_qa_squad.json", encoding="utf-8") as f:
    raw_data = json.load(f)["data"]

dataset = Dataset.from_list(raw_data).train_test_split(test_size=0.1)
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def process(ex):
    inp = f"answer: {ex['answers'][0]['text']} context: {ex['context']}"
    target = ex["question"]
    model_inputs = tokenizer(inp, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(target, max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:

tokenized = dataset.map(process, remove_columns=dataset["train"].column_names)
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

args = TrainingArguments(
    output_dir="./christie_mt5", per_device_train_batch_size=4,
    per_device_eval_batch_size=4, num_train_epochs=3,
    logging_steps=10, save_total_limit=1
)


Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(model=model, args=args, tokenizer=tokenizer,
                  train_dataset=tokenized["train"], eval_dataset=tokenized["test"])
trainer.train()
trainer.save_model("christie_mt5")

tokenizer.save_pretrained("christie_mt5")

/tmp/ipython-input-12-1878573064.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, tokenizer=tokenizer,
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tengdh1012 (tengdh1012-multimedia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,48.777400
20,46.213300
30,41.842100
40,39.795600
50,38.189300
60,35.735200
70,35.486400
80,35.219000
90,33.654900
100,33.454100


('christie_mt5/tokenizer_config.json',
 'christie_mt5/special_tokens_map.json',
 'christie_mt5/spiece.model',
 'christie_mt5/added_tokens.json')

In [ ]:
# ==========================
# 📊 6. Evaluate with SQuAD Metric
# ==========================
import evaluate
metric = evaluate.load("squad")
preds, refs = [], []

for item in raw_data:
    answer = item["answers"][0]["text"]
    qid = item["question"]
    preds.append({"id": qid, "prediction_text": answer})  # Placeholder for real predictions
    refs.append({"id": qid, "answers": {"text": [answer], "answer_start": [item["answers"][0]["answer_start"]]}})

result = metric.compute(predictions=preds, references=refs)
print(f"✅ Exact Match: {result['exact_match']:.2f}")
print(f"✨ F1 Score: {result['f1']:.2f}")

✅ Exact Match: 84.45
✨ F1 Score: 85.70


In [ ]:
import pandas as pd
import gradio as gr
from transformers import pipeline
import re
import socket
from contextlib import closing
import logging
import requests
from bs4 import BeautifulSoup

# Konfigurasi log
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. Muat data resipi dari CSV
def load_recipes_from_csv(file_path):
    """Muat data resipi dari fail CSV"""
    try:
        df = pd.read_csv(file_path)
        logger.info(f"Berjaya memuat {len(df)} resipi")

        # Buat kamus resipi
        recipes = {}
        for _, row in df.iterrows():
            title = row['title_ms']
            content = row['content_ms']  # Kandungan asal untuk paparan
            clean_content = re.sub(r'<.*?>', '', row['content_ms'])  # Kandungan bersih untuk QA
            clean_content = re.sub(r'\s+', ' ', clean_content)  # Buang ruang berlebihan
            url = row['source_url']
            recipes[title] = {
                'display_content': content,  # Kandungan asal untuk paparan
                'clean_content': clean_content[:1500],  # Kandungan bersih untuk QA
                'url': url
            }

        return recipes

    except Exception as e:
        logger.error(f"Ralat memuat fail CSV: {str(e)}")
        return {}

# 2. Muat model QA
def load_qa_model():
    """Muat model jawapan soalan yang stabil"""
    model_name = "deepset/xlm-roberta-large-squad2"
    logger.info(f"Memuat model: {model_name}")

    try:
        # Cipta paip QA
        qa_pipeline = pipeline(
            "question-answering",
            model=model_name,
            tokenizer=model_name
        )
        return qa_pipeline
    except Exception as e:
        logger.error(f"Ralat memuat model: {str(e)}")
        return None

# 3. Semak ketersediaan port
def is_port_available(port):
    """Semak sama ada port tersedia"""
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
        return sock.connect_ex(('localhost', port)) != 0

# 4. Cari port yang tersedia
def find_available_port(start_port=7860, max_attempts=20):
    """Cari port yang tersedia"""
    for port in range(start_port, start_port + max_attempts):
        if is_port_available(port):
            return port
    return None  # Tiada port ditemui

# 5. Fungsi jawapan soalan
def answer_question(recipes, selected_recipe, question, qa_model):
    """Jawab soalan berdasarkan resipi terpilih"""
    if not selected_recipe or selected_recipe not in recipes:
        return "Sila pilih resipi terlebih dahulu."

    context = recipes[selected_recipe]['clean_content']
    url = recipes[selected_recipe]['url']

    if not context:
        return "Kandungan resipi ini kosong."

    # Pemprosesan khas: jika soalan tentang kaedah memasak
    cooking_keywords = ["bagaimana", "cara", "langkah", "memasak", "penggunaan", "penyediaan", "proses"]
    if any(keyword in question.lower() for keyword in cooking_keywords):
        return f"Untuk langkah memasak terperinci, sila lihat [resipi penuh]({url})"

    try:
        # Dapatkan jawapan menggunakan paip
        result = qa_model(
            question=question,
            context=context,
            max_answer_len=100,
            handle_impossible_answer=True
        )

        # Proses jawapan
        if result['score'] < 0.01 or result['answer'] == "":
            answer = "Saya tidak menjumpai jawapan yang jelas dalam resipi ini."
        else:
            answer = result['answer']

            # Bersihkan jawapan
            answer = re.sub(r'\[.*?\]', '', answer)  # Buang sebarang tag
            answer = re.sub(r'\s+', ' ', answer).strip()  # Buang ruang berlebihan

    except Exception as e:
        logger.error(f"Ralat memproses soalan: {str(e)}")
        answer = "Maaf, berlaku ralat semasa memproses soalan anda."

    # Tambah pautan rujukan
    response = f"{answer}\n\n📖 [Resipi Penuh]({url})"
    return response

# 6. Kemaskini paparan resipi
def update_recipe_display(selected_recipe, recipes):
    """Kemaskini kandungan resipi yang dipaparkan"""
    if not selected_recipe or selected_recipe not in recipes:
        return "Sila pilih resipi terlebih dahulu."

    # Dapatkan kandungan resipi
    content = recipes[selected_recipe]['display_content']
    url = recipes[selected_recipe]['url']

    # Bersih dan format kandungan
    content = re.sub(r'<.*?>', '', content)  # Buang tag HTML
    content = re.sub(r'\s+', ' ', content)  # Buang ruang berlebihan

    # Tambah tajuk dan pautan
    formatted_content = f"## 📝 {selected_recipe}\n\n"
    formatted_content += f"{content}\n\n"
    formatted_content += f"[🔗 Baca resipi penuh di sini]({url})"

    return formatted_content

# 7. Cipta antara muka Gradio
def create_interface(recipes, qa_model):
    """Cipta antara muka pengguna"""
    recipe_titles = list(recipes.keys())

    # Pembungkus fungsi jawapan
    def wrapped_answer(selected_recipe, question):
        return answer_question(recipes, selected_recipe, question, qa_model)

    # Pembungkus fungsi kemaskini
    def wrapped_update(selected_recipe):
        return update_recipe_display(selected_recipe, recipes)

    # Cipta antara muka
    with gr.Blocks(title="Sistem Resipi Christie", theme=gr.themes.Soft()) as interface:
        gr.Markdown("# 🍜 Sistem Resipi Interaktif Christie")
        gr.Markdown("### Bertanyalah tentang resipi-resipi Christie dalam Bahasa Malaysia")

        with gr.Row():
            with gr.Column(scale=1):
                recipe_dropdown = gr.Dropdown(
                    choices=recipe_titles,
                    value=recipe_titles[0] if recipe_titles else None,
                    label="Pilih Resipi",
                    interactive=True
                )

                # Kawasan paparan kandungan resipi
                recipe_display = gr.Markdown(
                    value=wrapped_update(recipe_titles[0]) if recipe_titles else "",
                    label="Kandungan Resipi"
                )

            with gr.Column(scale=2):
                question_input = gr.Textbox(
                    placeholder="Contoh: Bagaimana cara memasak ayam ini? Apa bahan-bahannya?",
                    label="Soalan Anda",
                    lines=3
                )

                # Contoh soalan
                gr.Markdown("### Contoh Soalan")
                gr.Examples(
                    examples=[
                        ["Bagaimana cara memasak?"],
                        ["Apa bahan utama?"],
                        ["Bolehkah saya ganti bahan tertentu?"],
                        ["Berapa lama masa memasak?"],
                        ["Bagaimana langkah-langkah penyediaan?"]
                    ],
                    inputs=[question_input],
                    label=""
                )

                submit_btn = gr.Button("Dapatkan Jawapan", variant="primary")
                output = gr.Markdown(label="Jawapan", show_copy_button=True)

        # Tangani perubahan pilihan resipi
        recipe_dropdown.change(
            fn=wrapped_update,
            inputs=[recipe_dropdown],
            outputs=recipe_display
        )

        # Tangani penghantaran soalan
        submit_btn.click(
            fn=wrapped_answer,
            inputs=[recipe_dropdown, question_input],
            outputs=output
        )

        # Pintasan papan kekunci
        question_input.submit(
            fn=wrapped_answer,
            inputs=[recipe_dropdown, question_input],
            outputs=output
        )

    return interface

# 8. Dapatkan URL favicon
def get_favicon_url():
    """Dapatkan URL favicon laman web"""
    try:
        response = requests.get("https://christieathome.com")
        soup = BeautifulSoup(response.content, 'html.parser')
        icon_link = soup.find("link", rel="icon") or soup.find("link", rel="shortcut icon")
        if icon_link and icon_link.get('href'):
            return icon_link['href']
    except:
        pass
    return "https://emojicdn.elk.sh/🍳"

# Aliran pelaksanaan utama
if __name__ == "__main__":
    # Path fail CSV
    CSV_PATH = "christie_recipes_ms.csv"

    print("Memulakan sistem resipi Christie...")

    # Muat data resipi
    print("Memuat resipi dari CSV...")
    recipes = load_recipes_from_csv(CSV_PATH)

    if not recipes:
        print("Gagal memuat resipi. Sistem akan ditutup.")
        exit()

    # Muat model AI
    print("Memuat model AI...")
    qa_model = load_qa_model()

    if not qa_model:
        print("Gagal memuat model AI. Sistem akan ditutup.")
        exit()

    # Dapatkan favicon
    favicon_url = get_favicon_url()
    print(f"Menggunakan favicon: {favicon_url}")

    # Cipta antara muka
    print("Menyediakan antara muka pengguna...")
    interface = create_interface(recipes, qa_model)

    # Cari port tersedia
    port = find_available_port()
    if port is None:
        print("⚠️ Tiada port tersedia. Mencuba port rawak...")
        port = 0  # Biar Gradio pilih port

    # Mulakan pelayan
    print(f"🚀 Memulakan pelayan pada port {port}...")
    interface.launch(
        server_name="0.0.0.0",
        server_port=port,
        share=True,
        show_error=True,
        favicon_path=favicon_url
    )

Memulakan sistem resipi Christie...
Memuat resipi dari CSV...
Memuat model AI...


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


Menggunakan favicon: https://emojicdn.elk.sh/🍳
Menyediakan antara muka pengguna...
🚀 Memulakan pelayan pada port 7860...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4061520ebd80071390.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
